In [1]:
import os
import tensorflow as tf
import tensorflow_io as tfio
from keras.applications.efficientnet import EfficientNetB0
from keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy

In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
DATA_DIR = 'data'
HDF5_FILE = 'data_tf.hdf5'

In [4]:
BATCH_SIZE = 64
IMG_SHAPE = (96, 96, 4)

In [5]:
data_file = os.path.join(DATA_DIR, HDF5_FILE)
ds_x = tfio.IODataset.from_hdf5(data_file, '/x_train')
ds_y = tfio.IODataset.from_hdf5(data_file, '/y_train')

In [6]:
# convert to float and scale to [0, 1)
ds_x = ds_x.map(lambda x: tf.image.convert_image_dtype(x, tf.float32))
# ds_x = ds_images.map(lambda x: tf.image.per_image_standardization(x))

In [7]:
ds_xy = tf.data.Dataset.zip((ds_x, ds_y))
ds_xy = ds_xy.shuffle(BATCH_SIZE*2)

ds_len = ds_xy.reduce(0, lambda x, _: x + 1).numpy()
train_len = int(0.9 * ds_len)

ds_train = ds_xy.take(train_len)
ds_test = ds_xy.skip(train_len)

ds_len, train_len

(220025, 198022)

In [8]:
inputs = Input(shape=IMG_SHAPE)
model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=None)
model.trainable = True
x = GlobalAveragePooling2D()(model.output)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

# model.summary()

In [9]:
ds_train = ds_train.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-3),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
3095/3095 - 406s - loss: 0.3864 - accuracy: 0.8375 - val_loss: 0.6389 - val_accuracy: 0.7822
Epoch 2/10
3095/3095 - 391s - loss: 0.3719 - accuracy: 0.8416 - val_loss: 0.3453 - val_accuracy: 0.8616
Epoch 3/10
3095/3095 - 388s - loss: 0.2975 - accuracy: 0.8794 - val_loss: 0.3615 - val_accuracy: 0.8580
Epoch 4/10
3095/3095 - 389s - loss: 0.3018 - accuracy: 0.8809 - val_loss: 0.2988 - val_accuracy: 0.8810
Epoch 5/10
3095/3095 - 389s - loss: 0.2606 - accuracy: 0.8962 - val_loss: 0.2292 - val_accuracy: 0.9109
Epoch 6/10
3095/3095 - 389s - loss: 0.2595 - accuracy: 0.8982 - val_loss: 0.3150 - val_accuracy: 0.8682
Epoch 7/10
3095/3095 - 388s - loss: 0.2447 - accuracy: 0.9031 - val_loss: 0.3805 - val_accuracy: 0.8415
Epoch 8/10
3095/3095 - 389s - loss: 0.2363 - accuracy: 0.9084 - val_loss: 0.6090 - val_accuracy: 0.7898
Epoch 9/10
3095/3095 - 388s - loss: 0.2563 - accuracy: 0.8999 - val_loss: 0.2472 - val_accuracy: 0.8994
Epoch 10/10
3095/3095 - 388s - loss: 0.2184 - accuracy: 0.9148 -

In [10]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(128).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(128).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=2e-4),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
1548/1548 - 377s - loss: 0.2011 - accuracy: 0.9228 - val_loss: 0.1824 - val_accuracy: 0.9291
Epoch 2/10
1548/1548 - 370s - loss: 0.1775 - accuracy: 0.9326 - val_loss: 0.1741 - val_accuracy: 0.9341
Epoch 3/10
1548/1548 - 370s - loss: 0.1681 - accuracy: 0.9365 - val_loss: 0.1637 - val_accuracy: 0.9390
Epoch 4/10
1548/1548 - 372s - loss: 0.1608 - accuracy: 0.9392 - val_loss: 0.1678 - val_accuracy: 0.9361
Epoch 5/10
1548/1548 - 362s - loss: 0.1553 - accuracy: 0.9417 - val_loss: 0.1806 - val_accuracy: 0.9297
Epoch 6/10
1548/1548 - 373s - loss: 0.1482 - accuracy: 0.9440 - val_loss: 0.1781 - val_accuracy: 0.9316
Epoch 7/10
1548/1548 - 373s - loss: 0.1433 - accuracy: 0.9461 - val_loss: 0.1703 - val_accuracy: 0.9355
Epoch 8/10
1548/1548 - 373s - loss: 0.1392 - accuracy: 0.9485 - val_loss: 0.2069 - val_accuracy: 0.9186
Epoch 9/10
1548/1548 - 373s - loss: 0.1387 - accuracy: 0.9481 - val_loss: 0.1510 - val_accuracy: 0.9432
Epoch 10/10
1548/1548 - 373s - loss: 0.1303 - accuracy: 0.9517 -

In [11]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(256).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(256).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=5e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10
774/774 - 366s - loss: 0.1208 - accuracy: 0.9559 - val_loss: 0.1352 - val_accuracy: 0.9496
Epoch 2/10
774/774 - 356s - loss: 0.1192 - accuracy: 0.9559 - val_loss: 0.1944 - val_accuracy: 0.9279
Epoch 3/10
774/774 - 356s - loss: 0.1180 - accuracy: 0.9562 - val_loss: 0.1338 - val_accuracy: 0.9505
Epoch 4/10
774/774 - 353s - loss: 0.1161 - accuracy: 0.9572 - val_loss: 0.1337 - val_accuracy: 0.9503
Epoch 5/10
774/774 - 354s - loss: 0.1154 - accuracy: 0.9572 - val_loss: 0.1319 - val_accuracy: 0.9516
Epoch 6/10
774/774 - 357s - loss: 0.1139 - accuracy: 0.9583 - val_loss: 0.1323 - val_accuracy: 0.9507
Epoch 7/10
774/774 - 346s - loss: 0.1131 - accuracy: 0.9585 - val_loss: 0.1341 - val_accuracy: 0.9505
Epoch 8/10
774/774 - 345s - loss: 0.1114 - accuracy: 0.9594 - val_loss: 0.1319 - val_accuracy: 0.9517
Epoch 9/10
774/774 - 345s - loss: 0.1106 - accuracy: 0.9597 - val_loss: 0.1315 - val_accuracy: 0.9519
Epoch 10/10
774/774 - 345s - loss: 0.1095 - accuracy: 0.9601 - val_loss: 0.1330 - 

In [12]:
ds_train = ds_train.unbatch()
ds_test = ds_test.unbatch()
ds_train = ds_train.batch(512).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.batch(512).prefetch(tf.data.AUTOTUNE)

model.compile(loss=BinaryCrossentropy(from_logits=False),
              optimizer=Adam(learning_rate=1e-5),
              metrics=['accuracy'])

history = model.fit(ds_train,
                    epochs=10,
                    verbose=2, 
                    validation_data=ds_test
                   )

Epoch 1/10


ResourceExhaustedError:  OOM when allocating tensor with shape[512,672,6,6] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/block5b_expand_activation/Sigmoid (defined at tmp/ipykernel_7184/1563839495.py:10) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_178391]

Function call stack:
train_function
